# ***NOTEBOOK EN GBQ: "Cargas de trabajo - etl_tp"***

# INICIALIZACIÓN

In [42]:
# @title INSTALACIÓN DE LIBRERÍAS

# !pip install google-cloud-secret-manager


In [43]:
# @title IMPORTACIÓN DE LIBRERÍAS GCP

from google.cloud import storage
import os

from google.cloud import bigquery  # Importa el cliente de BigQuery
from google.cloud import storage   # Importa el cliente de Cloud Storage
import pandas as pd
import io
import re
import unicodedata
import chardet  # Biblioteca para detectar codificaciones de texto

from google.colab import auth
auth.authenticate_user()

from google.cloud import secretmanager
import os

# Configura el cliente de Secret Manager
client = secretmanager.SecretManagerServiceClient()
project_id = "animum-dev-datawarehouse"

# Función para obtener un secreto por su nombre
def get_secret(secret_id):
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

# Obtén los valores de los secretos
hs_datawarehouse_acces_token = get_secret("hs_datawarehouse_acces_token")
hs_datawarehouse_secret_key = get_secret("hs_datawarehouse_secret_key")

# Configura las variables de entorno
os.environ['hs_datawarehouse_acces_token'] = hs_datawarehouse_acces_token
os.environ['hs_datawarehouse_secret_key'] = hs_datawarehouse_secret_key

print("Secretos configurados correctamente.")

Secretos configurados correctamente.


In [89]:
# @title HS_to_GBQ_sensitive_data()
def HS_to_GBQ_sensitive_data(params: dict):
    """
    Extrae contactos de HubSpot en chunks de hasta 10k contactos cada uno,
    filtrando por un rango de 'createdate', y sube cada chunk a BigQuery.

    Args:
        params (dict):
            - HS_api_key (str): API key de HubSpot o token de Private App.
            - GBQ_project_id (str): ID del proyecto de BigQuery.
            - GCS_bucket_name (str): Nombre del bucket de GCS.
            - GBQ_dataset_id (str): Dataset de BigQuery.
            - GBQ_table_id (str): Tabla de destino en BigQuery.
            - HS_fields_no_sensitive_names_list (list): Lista de campos NO sensibles a recuperar
              de HubSpot (p.ej., ["email"]). Se utilizarán para paginar; se forzarán "id" y "createdate".
            - HS_fields_sensitive_names_list (list): Lista de campos SENSIBLES a recuperar vía batch/read
              (p.ej., ["iban", "codigo_bic_swift", "documento_nacional_de_identidad_numero"]).
            - HS_api_lines_per_call (int): Límite de registros por llamada a la API de HubSpot.
            - hs_contact_filter_createdate (dict): Filtro de fechas, por ejemplo:
              {"from": "2024-01-01", "to": "2025-02-02", "mode": "between"}.

    Returns:
        None
    """

    import requests
    import pandas as pd
    from google.colab import auth
    from google.cloud import bigquery, storage
    from datetime import datetime, timedelta
    import os
    import uuid

    # ========== LECTURA DE PARÁMETROS ==========
    HS_api_key = params["HS_api_key"]
    GBQ_project_id = params["GBQ_project_id"]
    GCS_bucket_name = params["GCS_bucket_name"]
    GBQ_dataset_id = params["GBQ_dataset_id"]
    GBQ_table_id = params["GBQ_table_id"]
    HS_api_lines_per_call = params["HS_api_lines_per_call"]
    createdate_filter = params["hs_contact_filter_createdate"]

    hs_fields_no_sensitive = params["HS_fields_no_sensitive_names_list"]
    hs_fields_sensitive = params["HS_fields_sensitive_names_list"]

    # Forzamos que "id" y "createdate" estén en la parte no sensible (son obligatorios)
    mandatory = {"id", "createdate"}
    for field in mandatory:
        if field not in hs_fields_no_sensitive:
            hs_fields_no_sensitive.append(field)

    # Convertir las fechas a datetime; 'to' se ajusta a las 23:59:59
    from_date = datetime.strptime(createdate_filter["from"], "%Y-%m-%d")
    to_date = datetime.strptime(createdate_filter["to"], "%Y-%m-%d") + timedelta(days=1, seconds=-1)

    non_sens = sorted(hs_fields_no_sensitive)
    sens = sorted(hs_fields_sensitive)

    print("=== Comenzando extracción de contactos HubSpot ===")
    print(f"Parámetros de ejecución:\n"
          f"  - Rango de fechas: {from_date} a {to_date}\n"
          f"  - Campos no sensibles (para búsqueda): {non_sens}\n"
          f"  - Campos sensibles (batch read): {sens}\n")

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FUNCIÓN: BUSCAR CONTACTOS (SIN CAMPOS SENSIBLES) POR RANGO DE FECHAS
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    def hubspot_search_chunk(start_date, end_date, limit_chunk=10000):
        print(f"   [INFO] Extrayendo contactos desde {start_date.isoformat()} hasta {end_date.isoformat()}. Límite total: {limit_chunk}")
        start_iso = start_date.isoformat(timespec="seconds") + "Z"
        end_iso = end_date.isoformat(timespec="seconds") + "Z"
        url_search = "https://api.hubapi.com/crm/v3/objects/contacts/search"
        headers = {"Authorization": f"Bearer {HS_api_key}", "Content-Type": "application/json"}
        accumulated = []
        after = None
        while len(accumulated) < limit_chunk:
            remaining = limit_chunk - len(accumulated)
            body = {
                "filterGroups": [
                    {
                        "filters": [
                            {"propertyName": "createdate", "operator": "GTE", "value": start_iso},
                            {"propertyName": "createdate", "operator": "LTE", "value": end_iso},
                        ]
                    }
                ],
                "sorts": [{"propertyName": "createdate", "direction": "ASCENDING"}],
                "properties": non_sens,
                "limit": min(HS_api_lines_per_call, remaining),
            }
            if after:
                body["after"] = after
            resp = requests.post(url_search, headers=headers, json=body)
            if resp.status_code != 200:
                raise requests.HTTPError(f"❌ Error en /search: {resp.text}")
            data = resp.json()
            results = data.get("results", [])
            if not results:
                break
            for r in results:
                props = r.get("properties", {})
                row = {}
                for c in non_sens:
                    if c == "id":
                        row[c] = r.get("id")  # Extraer 'id' del nivel superior
                    else:
                        row[c] = props.get(c)
                accumulated.append(row)
            after = data.get("paging", {}).get("next", {}).get("after")
            if not after:
                break
        if not accumulated:
            return [], None, False
        last_cdate_str = accumulated[-1].get("createdate")
        if last_cdate_str is not None:
            try:
                last_cdate_dt = datetime.fromisoformat(last_cdate_str.replace("Z", ""))
            except ValueError:
                last_cdate_dt = None
        else:
            last_cdate_dt = None
        reached = (len(accumulated) >= limit_chunk)
        return accumulated, last_cdate_dt, reached

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FUNCIÓN: BATCH READ DE CAMPOS SENSIBLES DADO UNA LISTA DE CONTACTOS
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    def hubspot_batch_sens(contact_rows):
        import math
        if not sens:
            print("   [INFO] No se han solicitado campos sensibles. Saltamos batch read.")
            return pd.DataFrame(contact_rows)
        if not contact_rows:
            print("   [INFO] Lista vacía de contactos. Saltamos batch read.")
            return pd.DataFrame([])
        ids = [x.get("id") for x in contact_rows if x.get("id")]
        if not ids:
            print("   [WARNING] Ningún contacto con 'id'. Saltamos batch read.")
            return pd.DataFrame(contact_rows)
        print(f"   [INFO] Recuperando {len(sens)} propiedades sensibles para {len(ids)} contactos vía batch/read.")
        chunk_size = 100
        total_ids = len(ids)
        n_chunks = math.ceil(total_ids / chunk_size)
        url_batch = "https://api.hubapi.com/crm/v3/objects/contacts/batch/read"
        headers = {"Authorization": f"Bearer {HS_api_key}", "Content-Type": "application/json"}
        id_to_sens = {}
        for i in range(n_chunks):
            subset_ids = ids[i*chunk_size : (i+1)*chunk_size]
            body_b = {
                "properties": sens,
                "inputs": [{"id": cid} for cid in subset_ids]
            }
            resp_b = requests.post(url_batch, headers=headers, json=body_b)
            if resp_b.status_code != 200:
                raise requests.HTTPError(f"❌ Error en batch/read: {resp_b.text}")
            data_b = resp_b.json()
            results_b = data_b.get("results", [])
            for rb in results_b:
                c_id = rb.get("id")
                p_b = rb.get("properties", {})
                id_to_sens[c_id] = {s: p_b.get(s) for s in sens}
            print(f"      - Chunk {i+1}/{n_chunks} => {len(results_b)} contactos OK.")
        df_no_sens = pd.DataFrame(contact_rows)
        for sfield in sens:
            df_no_sens[sfield] = df_no_sens["id"].apply(lambda c_id: id_to_sens.get(c_id, {}).get(sfield))
        return df_no_sens

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FUNCIÓN: SUBIR A BIGQUERY (USANDO CSV TEMPORAL EN GCS)
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    def upload_df_to_bq(df, disposition="WRITE_APPEND"):
        if df.empty:
            print("   [WARNING] DataFrame vacío, no se subirá nada.")
            return
        auth.authenticate_user()
        print(f"   [INFO] Subiendo DataFrame con {len(df)} filas a BigQuery (disposition={disposition}).")
        temp_file_local = "temp_contacts_chunk.csv"
        df.to_csv(temp_file_local, index=False)
        storage_client = storage.Client(project=GBQ_project_id)
        bucket = storage_client.bucket(GCS_bucket_name)
        blob_name = f"temp_contacts_{uuid.uuid4().hex}.csv"
        blob = bucket.blob(blob_name)
        print(f"      - Subiendo CSV a gs://{GCS_bucket_name}/{blob_name} ...")
        blob.upload_from_filename(temp_file_local)
        gcs_uri = f"gs://{GCS_bucket_name}/{blob_name}"
        bq_client = bigquery.Client(project=GBQ_project_id)
        table_id = f"{GBQ_project_id}.{GBQ_dataset_id}.{GBQ_table_id}"
        job_config = bigquery.LoadJobConfig(
            write_disposition=disposition,
            source_format=bigquery.SourceFormat.CSV,
            autodetect=True,
        )
        try:
            print(f"      - Cargando en BQ => {table_id} ...")
            load_job = bq_client.load_table_from_uri(gcs_uri, table_id, job_config=job_config)
            load_job.result()
            print(f"      ✅ Carga completada. Se subieron {len(df)} filas.")
        finally:
            blob.delete()
            if os.path.exists(temp_file_local):
                os.remove(temp_file_local)

    # ========== LÓGICA PRINCIPAL: DIVIDIR EN CHUNKS ==========
    chunk_index = 0
    current_start = from_date
    print("=== Iniciando loop de extracción por chunks de hasta 10k contactos ===")
    while current_start <= to_date:
        chunk_index += 1
        print(f"=== Procesando chunk #{chunk_index} ===")
        contacts, last_cdate, reached_limit = hubspot_search_chunk(current_start, to_date)
        if not contacts:
            print("   [INFO] No se encontraron más contactos en este sub-rango.")
            break
        print(f"   [INFO] {len(contacts)} contactos extraídos en este chunk. Iniciando batch read de campos sensibles...")
        df_chunk = hubspot_batch_sens(contacts)
        disposition = "WRITE_TRUNCATE" if chunk_index == 1 else "WRITE_APPEND"
        upload_df_to_bq(df_chunk, disposition=disposition)
        if not last_cdate:
            print("   [WARNING] No se pudo determinar la última fecha 'createdate'. Terminando...")
            break
        next_start = last_cdate + timedelta(seconds=1)
        current_start = next_start
        if not reached_limit:
            print("   [INFO] No se alcanzó el límite de 10k. No hay más contactos en el rango.")
            break
    print("=== Extracción y carga completadas exitosamente. ===")


# EJECUCIONES

In [90]:
# @title IMPORTACIÓN DATOS SENSIBLES HS TO GBQ
hs_contact_filter_createdate_from = "2024-01-01"  # @param {"type":"date"}
hs_contact_filter_createdate_to   = "2025-02-02"  # @param {"type":"date"}

params = {
    "HS_api_key": hs_datawarehouse_acces_token,  # Tu token o API key de HubSpot
    "GBQ_project_id": "animum-dev-datawarehouse",  # ID de tu proyecto en BigQuery
    "GCS_bucket_name": "temp_datawarehouse",         # Nombre del bucket en GCS para archivos temporales
    "GBQ_dataset_id": "tp_02st_01",                   # Dataset en BigQuery
    "GBQ_table_id": "hs_contact_sensitive_cleaned",   # Tabla destino en BigQuery
    "HS_fields_no_sensitive_names_list": [           # Lista de campos NO sensibles (obligatoriamente debe incluir "id" y "createdate")
        "email"                                      # Ejemplo: se puede incluir "email"
    ],
    "HS_fields_sensitive_names_list": [              # Lista de campos SENSIBLES a recuperar vía batch/read
        "iban",
        "codigo_bic_swift",
        "documento_nacional_de_identidad_numero"
    ],
    "HS_api_lines_per_call": 100,                     # Número de registros por llamada a la API de HubSpot
    "hs_contact_filter_createdate": {                 # Filtro de fechas para la extracción
        "from": hs_contact_filter_createdate_from,                         # Fecha de inicio (YYYY-MM-DD)
        "to": hs_contact_filter_createdate_to,                           # Fecha de fin (YYYY-MM-DD)
        "mode": "between"
    }
}


HS_to_GBQ_sensitive_data(params)


=== Comenzando extracción de contactos HubSpot ===
Parámetros de ejecución:
  - Rango de fechas: 2024-01-01 00:00:00 a 2025-02-02 23:59:59
  - Campos no sensibles (para búsqueda): ['createdate', 'email', 'id']
  - Campos sensibles (batch read): ['codigo_bic_swift', 'documento_nacional_de_identidad_numero', 'iban']

=== Iniciando loop de extracción por chunks de hasta 10k contactos ===
=== Procesando chunk #1 ===
   [INFO] Extrayendo contactos desde 2024-01-01T00:00:00 hasta 2025-02-02T23:59:59. Límite total: 10000
   [INFO] 10000 contactos extraídos en este chunk. Iniciando batch read de campos sensibles...
   [INFO] Recuperando 3 propiedades sensibles para 10000 contactos vía batch/read.
      - Chunk 1/100 => 100 contactos OK.
      - Chunk 2/100 => 100 contactos OK.
      - Chunk 3/100 => 100 contactos OK.
      - Chunk 4/100 => 100 contactos OK.
      - Chunk 5/100 => 100 contactos OK.
      - Chunk 6/100 => 100 contactos OK.
      - Chunk 7/100 => 100 contactos OK.
      - Chunk 8